In [2]:
# Import necessary modules
import data_preprocessor as dp
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load the dataset
messy_data = pd.read_csv('../Data/messy_data.csv')
clean_data = messy_data.copy()

print(messy_data.describe())
print(messy_data.info())
print(messy_data.head())

print("After loading:", type(clean_data))
# Display dataset information


# 2. Preprocess the data
clean_data = dp.impute_missing_values(clean_data, strategy='mean')
print("After impute:", type(clean_data))

# Drop columns that still have missing values
clean_data = clean_data.dropna(axis=1)
print("After dropna:", type(clean_data))

# 1. Remove duplicate rows
clean_data = dp.remove_duplicates(clean_data)
print("After remove_duplicates:", type(clean_data))

# 2. Check for duplicate columns
duplicates = clean_data.T.duplicated()
redundant_cols = clean_data.columns[duplicates]
print("duplicate columns:", list(redundant_cols))
# Remove duplicate columns
clean_data = clean_data.loc[:, ~clean_data.T.duplicated()]

# to check numerical columns for normalization
numeric_cols = clean_data.select_dtypes(include=['number'])

if not numeric_cols.empty:
    print("Range of each numeric column:")
    print(numeric_cols.agg(['min', 'max']))
else:
    print("No numeric columns to normalize.")

# Normalize only numeric columns and update clean_data
numeric_cols = clean_data.select_dtypes(include=[np.number]).columns
normalized = dp.normalize_data(clean_data[numeric_cols])
# Ensure normalized is a DataFrame with correct columns and index
if not isinstance(normalized, pd.DataFrame):
    normalized = pd.DataFrame(normalized, columns=numeric_cols, index=clean_data.index)
clean_data.loc[:, numeric_cols] = normalized
print("After normalize_data:", type(clean_data))

# check for outliers using Z-score method on numeric columns only
outliers = np.abs(stats.zscore(clean_data[numeric_cols])) > 3
print("Outliers (threshold=3):", outliers.sum())

# scatter plot to visualize outliners 
clean_data['outlier'] = outliers.any(axis=1)

# Plot
sns.scatterplot(data=clean_data, 
                x=feature_cols[0], 
                y=feature_cols[1],
                hue='outlier', palette=['blue', 'red'])
plt.title('Scatter Plot to Identify Outliers')
plt.xlabel(feature_cols[0])
plt.ylabel(feature_cols[1])
plt.show()

# (Optional: remove or impute outliers here)
clean_data = clean_data.loc[~outliers.any(axis=1)]

print("Before removal:", clean_data.columns)
# Remove redundant features 
clean_data = dp.remove_redundant_features(clean_data)
print("After removal:", clean_data.columns)

# Ensure target is binary (0 or 1)
clean_data['target'] = (clean_data['target'] >= 0.5).astype(int)

# 3. Save the cleaned dataset
clean_data.to_csv('../Data/clean_data.csv', index=False)


#4. Train and evaluate the model
dp.simple_model(clean_data)





           target            b            c            f            h  \
count  920.000000  1196.000000  1196.000000  1196.000000  1158.000000   
mean     0.553261     0.004015    53.383779     0.001391   200.583765   
std      0.497426     0.984837     9.534033     1.011656   110.061582   
min      0.000000    -3.308750    28.000000    -2.820047     0.000000   
25%      0.000000    -0.692960    47.000000    -0.633350   176.250000   
50%      1.000000     0.004241    54.000000    -0.051965   224.000000   
75%      1.000000     0.714572    60.000000     0.693539   270.000000   
max      1.000000     2.982511    77.000000     3.323155   603.000000   

                j            k            l            n            o  \
count  591.000000  1196.000000  1196.000000  1012.000000  1008.000000   
mean    -0.600216     0.007269    -0.013268     4.904847   137.780754   
std      1.067309     2.498874     0.958741     0.200283    26.175380   
min     -5.954661   -24.268796    -2.802329     4.

c:\Users\ynaveed\Documents\BINF-5507-materials\Assignment1\Scripts\data_preprocessor.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
c:\Users\ynaveed\Documents\BINF-5507-materials\Assignment1\Scripts\data_preprocessor.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are

Range of each numeric column:
     target         b   c         f      h          k         l         n  \
min     0.0 -3.308750  28 -2.820047    0.0 -24.268796 -2.802329  4.094345   
max     1.0  2.982511  77  3.323155  603.0  27.315031  3.328148  5.273000   

         o           p           q          r    t          u      v  \
min   60.0   60.522848   60.035804  27.548190 -2.6  28.743286    0.0   
max  202.0  200.680509  202.572685  77.484697  6.2  78.520864  200.0   

            w          y  
min -3.087673  26.549257  
max  3.933288  79.712348  
After normalize_data: <class 'pandas.core.frame.DataFrame'>
Outliers (threshold=3): 81


c:\Users\ynaveed\Documents\BINF-5507-materials\Assignment1\Scripts\data_preprocessor.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_cols] = scaler.fit_transform(data[numeric_cols])
C:\Users\ynaveed\AppData\Local\Temp\ipykernel_8944\631399543.py:57: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.71428571 0.79591837 0.79591837 ... 0.46938776 0.69387755 0.36734694]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  clean_data.loc[:, numeric_cols] = normalized


NameError: name 'feature_cols' is not defined